# KGTK BROWSER Setup

We will setup the files required for KGTK Browser command to run. KGTK Browser provides an interface to browse your knowledge graph.

### Get the latest kgtk-browser from GitHub

In [ ]:
!rm -rf kgtk-browser && git clone https://github.com/usc-isi-i2/kgtk-browser
!pip install -r kgtk-browser/requirements.txt

### Install papermill to run a Jupyter notebook via command line

In [ ]:
!pip install papermill

### Import required modules

In [3]:
from kgtk.functions import kgtk, kypher
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
import json
import os
from google.colab.output import eval_js

### Setup Input Parameters

In [ ]:
files = 'label,pagerank_undirected,alias,description,claims,datatypes,qualifiers'.split(",")

input_files_url="https://github.com/usc-isi-i2/kgtk-tutorial-files/raw/main/datasets/kgtk-browser"

input_path = None
output_path = '/tmp/projects'
project_name = 'kgtk-browser'


ck = ConfigureKGTK(files, input_files_url=input_files_url)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name)


Take a look at the environment variables

In [5]:
ck.print_env_variables()

GRAPH: /Users/amandeep/isi-kgtk-tutorial/kgtk-browser_input
TEMP: /tmp/projects/kgtk-browser/temp.kgtk-browser
STORE: /tmp/projects/kgtk-browser/temp.kgtk-browser/wikidata.sqlite3.db
kypher: kgtk query --graph-cache /tmp/projects/kgtk-browser/temp.kgtk-browser/wikidata.sqlite3.db
USE_CASES_DIR: /Users/amandeep/Github/kgtk-notebooks/use-cases
KGTK_OPTION_DEBUG: false
EXAMPLES_DIR: /Users/amandeep/Github/kgtk-notebooks/examples
KGTK_LABEL_FILE: /Users/amandeep/isi-kgtk-tutorial/kgtk-browser_input/labels.en.tsv.gz
OUT: /tmp/projects/kgtk-browser
KGTK_GRAPH_CACHE: /tmp/projects/kgtk-browser/temp.kgtk-browser/wikidata.sqlite3.db
kgtk: kgtk
label: /Users/amandeep/isi-kgtk-tutorial/kgtk-browser_input/labels.en.tsv.gz
pagerank_undirected: /Users/amandeep/isi-kgtk-tutorial/kgtk-browser_input/metadata.pagerank.undirected.tsv.gz
alias: /Users/amandeep/isi-kgtk-tutorial/kgtk-browser_input/aliases.en.tsv.gz
description: /Users/amandeep/isi-kgtk-tutorial/kgtk-browser_input/descriptions.en.tsv.gz
cla

### Setup Cache file for KGTK Browser

In [6]:
!papermill kgtk-browser/KGTK-Query-Text-Search-Setup.ipynb kgtk-browser/KGTK-Query-Text-Search-Setup-OUT.ipynb \
-p input_path $GRAPH \
-p output_path $OUT \
-p project_name kgtk-browser-files \
-p create_db yes \
-p create_es no \
-p create_class_viz no 

Input Notebook:  kgtk-browser/KGTK-Query-Text-Search-Setup.ipynb
Output Notebook: kgtk-browser/KGTK-Query-Text-Search-Setup-OUT.ipynb
Black is not installed, parameters wont be formatted
Executing: 100%|██████████████████████████████| 64/64 [01:48<00:00,  1.69s/cell]


In [7]:
os.environ['STORE']=f"{os.environ['OUT']}/kgtk-browser-files/temp.kgtk-browser-files/wikidata.sqlite3.db"

### Lets take a look at the tables in the Cache

In [8]:
!kgtk query --gc $STORE --show-cache

Graph Cache:
DB file: /tmp/projects/kgtk-browser/kgtk-browser-files/temp.kgtk-browser-files/wikidata.sqlite3.db
  size:  526.05 MB   	free:  0 Bytes   	modified:  2022-03-23 11:32:59

KGTK File Information:
alias:
  size:  1.28 MB   	modified:  2022-03-23 11:31:24   	graph:  graph_3
claims:
  size:  31.25 MB   	modified:  2022-03-23 11:31:28   	graph:  graph_5
datatypes:
  size:  20.69 KB   	modified:  2022-03-23 11:31:29   	graph:  graph_6
description:
  size:  1.28 MB   	modified:  2022-03-23 11:31:25   	graph:  graph_4
l_d_pgr_ud:
  size:  3.04 MB   	modified:  2022-03-23 11:32:19   	graph:  graph_8
label:
  size:  1.02 MB   	modified:  2022-03-23 11:31:21   	graph:  graph_1
pagerank_undirected:
  size:  13.45 MB   	modified:  2022-03-23 11:31:22   	graph:  graph_2
qualifiers:
  size:  8.10 MB   	modified:  2022-03-23 11:31:31   	graph:  graph_7

Graph Table Information:
graph_1:
  size:  5.96 MB   	created:  2022-03-23 11:31:47
  header:  ['node1', 'label', 'node2', 'id']
graph_2:


In [9]:
os.environ['KGTK_BROWSER_GRAPH_CACHE'] = os.environ['STORE']

In [ ]:
# Get a unique link pointing to this google colab environment

print(eval_js("google.colab.kernel.proxyPort(8000)") + "browser")

In [ ]:
# Set the environment variables for flask
os.environ['FLASK_ENV'] = 'production'
os.environ['FLASK_APP'] = 'kgtk-browser/kgtk_browser_app.py'
os.environ['KGTK_BROWSER_CONFIG'] = 'kgtk-browser/browser/backend/kgtk_browser_config.py'

# Run the kgtk-browser app
!flask run --port 8000